# Модель
Модель предназначена для распознавания расфокуса или размытия изображений.


# Dataset:

*   https://www.kaggle.com/kwentar/blur-dataset



# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import math
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

!pip install tensorflow_addons
import tensorflow_addons as tfa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 14.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Добавляем каггл-датасеты

In [ ]:
! pip install -q kaggle

import os
from google.colab import files

if not os.path.exists("/root/.kaggle/kaggle.json"):
  files.upload()
  ! mkdir -p ~/.kaggle
  ! mv kaggle.json ~/.kaggle/ 
  ! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# example downloading dataset
!kaggle datasets download -d kwentar/blur-dataset
!unzip -qq blur-dataset.zip -d m_kotenkov_blur_dataset

100% 1.49G/1.49G [00:11<00:00, 91.9MB/s]
100% 1.49G/1.49G [00:11<00:00, 135MB/s] 


# Разделим blur_dataset

In [ ]:
blur_dir = "/content/m_kotenkov_blur_dataset/blur_dataset_scaled"
batch_size = 32
img_height = 128
img_width = 128
train_blur = keras.utils.image_dataset_from_directory(blur_dir,
                                                      validation_split=0.2,
                                                      subset="training",
                                                      seed=0,
                                                      image_size=(img_height, img_width),
                                                      batch_size=batch_size)
val_blur = keras.utils.image_dataset_from_directory(blur_dir,
                                                    validation_split=0.2,
                                                    subset="validation",
                                                    seed=0,
                                                    image_size=(img_height, img_width),
                                                    batch_size=batch_size)

Found 1050 files belonging to 3 classes.
Using 840 files for training.
Found 1050 files belonging to 3 classes.
Using 210 files for validation.


In [ ]:
class_names = train_blur.class_names
print(class_names)

for image_batch, labels_batch in train_blur:
  print(image_batch.shape)
  break


['defocused_blurred', 'motion_blurred', 'sharp']
(32, 256, 256, 3)


# Стандартизируем данные

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_blur.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

print(np.min(first_image), np.max(first_image))

0.0 1.0


# Задаём параметры модели и обучаем её

In [ ]:
input_shape = (4, 128, 128, 3)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(27, 3, activation='relu', input_shape=input_shape[1:]),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(48, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Conv2D(64, 3, activation='relu'),
    # tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names))
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(normalized_ds, epochs=10)

Epoch 1/10
27/27 [==============================] - 124s 4s/step - loss: 0.8259 - accuracy: 0.5917
Epoch 2/10
27/27 [==============================] - 109s 4s/step - loss: 0.7610 - accuracy: 0.6238
Epoch 3/10
27/27 [==============================] - 113s 4s/step - loss: 0.6793 - accuracy: 0.6357
Epoch 4/10
27/27 [==============================] - 108s 4s/step - loss: 0.6649 - accuracy: 0.6690
Epoch 5/10
27/27 [==============================] - 118s 4s/step - loss: 0.5285 - accuracy: 0.7583
Epoch 6/10
27/27 [==============================] - 109s 4s/step - loss: 0.4590 - accuracy: 0.8167
Epoch 7/10
27/27 [==============================] - 110s 4s/step - loss: 0.2981 - accuracy: 0.8643
Epoch 8/10
27/27 [==============================] - 109s 4s/step - loss: 0.2130 - accuracy: 0.9167
Epoch 9/10
27/27 [==============================] - 118s 4s/step - loss: 0.1856 - accuracy: 0.9310
Epoch 10/10
27/27 [==============================] - 111s 4s/step - loss: 0.1130 - accuracy: 0.9583


# Evaluation

In [ ]:
normalized_val_ds = val_blur.map(lambda x, y: (normalization_layer(x), y))

test_loss, test_acc = model.evaluate(normalized_val_ds, verbose=2)

print('\nTest accuracy:', test_acc)

7/7 - 18s - loss: 1.0487 - accuracy: 0.6762 - 18s/epoch - 3s/step

Test accuracy: 0.6761904954910278
